<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/Extract_CNIS_MackprevIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

<br>
<br>


# **MACKPrev IA. Extração de Informações do CNIS**
---

Execute e Selecione o arquivo de CNIS a ser processado em seu computador. Este programa fornecerá um sumário do seu extrato e arquivos com dados do beneficiário, vínculos e remunerações para facilitar a sua análise ou de um especialista.


<br>
<br>


In [ ]:
#@title $\leftarrow$ Apenas aperte o botão ;-)

#-------------------------------------------------------------------------------
# install dependencies routines
#-------------------------------------------------------------------------------

print('... Instalando as dependências. Isso pode demorar alguns minutos.')

!pip install colorama
!pip install pdflib

import colorama
from colorama import Fore, Style

print('\n')
print(Fore.BLUE + '... Dependências instaladas com sucesso.')
# print(Style.RESET_ALL)

#-------------------------------------------------------------------------------
# Upload CNIS File
#-------------------------------------------------------------------------------

print('\n')
print(Fore.BLUE + '... Faça o upload do seu CNIS em formado .pdf.')
print('\n')
# print(Style.RESET_ALL)

from google.colab import files

uploaded = files.upload()

print()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
#-------------------------------------------------------------------------------
# Start Processing
#-------------------------------------------------------------------------------

import re
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pdflib import Document

# !pip install db-sqlite3
# from google.colab import drive
# drive.mount('/content/drive')

#doc = Document("/content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS-Cristina Mitiyo Maru Inagaki.pdf")

doc = Document("/content/" + fn)

print('\n')
print(Fore.BLUE + '... Processando o arquivo: ' + fn)
print('\n')
print(Style.RESET_ALL)


#-------------------------------------------------------------------------------
# Processa Beneficiário
#-------------------------------------------------------------------------------


def get_client(page): 
  client = {
      "Nome": page[5].split("Nome: ", 1)[1],
      "NIT": page[5].split("NIT: ", 1)[1][:14],
      "CPF": page[5].split("CPF: ", 1)[1][:14],
      "Nome da mãe": page[6].split("Nome da mãe: ", 1)[1],
      "Data de nascimento": page[6].split("Data de nascimento: ", 1)[1][:10],
  }

  client = pd.DataFrame.from_records(client, index=[0])
  client["Data de criação do arquivo"] = page[3].split("Extrato Previdenciário")[1].strip()
  #Helaine e Júlia
  # TODO: Após ler o Sexo da tela, inserir o Sexo
  #client.insert(0,  "Sexo", "M")
  #client["Sexo"] = client["Sexo"].ffill()

  client.head()
  return client

#-------------------------------------------------------------------------------
# Funções Auxilares
#-------------------------------------------------------------------------------

def check_empty_values(val: list) -> list:
    for i in range(5, len(val)):

        # Data Fim
        if i == 5 and "/" not in val[i]:
            val.insert(i, None)

        # Últ. Remun.
        elif i == 7:
            if "/" not in val[i]:
                val.insert(i, None)

    # Indicadores e/ou Últ. Remun. se for preciso
    while len(val) < 9:
        val.append(None)

    val.append([])
    return val

# Substitui espaços em brancos maiores que 1 pelo caracter &
def remove_blank_spaces(line: str) -> str:
    return re.sub('  +', '&', line)

# Retorna o index da coluna dada a palavra
def get_colum_index(page: list, var: str) -> int:
    for line in page:
        if var in line:
            return page.index(line)

    return -1

def check_empty_values_rem(val: list) -> list:

    # For loop com entre os valores das remunerações
    for index in range(0, 9, 3):

        if index < len(val):  # Verificando a existência dos dados

            if val[index][2] == '/':  # Verificando a existência da data

                if val[index + 1][-3] == ',':  # Verificando a existência da remuneração

                    if index + 2 < len(val):  # Verificando a existência dos dados

                        if val[index + 2][
                                2] == '/':  # Verificando a existência dos indicadores - LBYL
                            val.insert(index + 2, None)

                    else:  # Inserindo o valor vazio caso os dados sejam inexistentes
                        val.append(None)

        else:
            for i in range(3):
                val.append(None)

    return val  # Retornando a lista

# Função que normaliza o json
def rem_list_to_json_normalized(indexes: list, val: list, seq: str) -> list:
    rem_dict = {}
    rem_list = []

    for i in range(0, 9, 3):
        if val[i]:
            rem_dict = {
                index: value
                for index, value in zip(indexes[:3], val[i:i + 3])
            }
            rem_dict["Seq."] = seq
            rem_list.append(rem_dict)

    return rem_list

# Função para normalizar as remunerações
def clean_up_data_normalized(dados: list, linha: list, df_relacao, df_remuneracao) -> None:
    global seq, rem_final

    line = remove_blank_spaces(linha).split("&")  # Pegando a linha

    if line[0].isnumeric(
    ):  # Verificando se o primeiro valor da linha é numérico
        values: list = check_empty_values(line)
        seq = line[0]

        result: dict = {index: value for index, value in zip(indexes_rel, values)}
        #print(result)
        df_relacao.append(pd.DataFrame.from_records(result, index=[0]))

    else:
        rem: list = check_empty_values_rem(line)
        rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

        dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
        
        df_remuneracao.append(pd.concat(dfs, ignore_index=True))

def clean_data(seq, linha, df_remuneracao):
  line = remove_blank_spaces(linha).split("&")

  if line[0].isnumeric():
      seq = line[0]

  else:

    rem: list = check_empty_values_rem(line)
    rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

    dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
    
    df_remuneracao.append(pd.concat(dfs, ignore_index=True))  

  return seq

  # Function to insert row in the dataframe
def insert_row(row_number, df, row_value):
  # Starting value of upper half
  start_upper = 0
  
  # End value of upper half
  end_upper = row_number
  
  # Start value of lower half
  start_lower = row_number
  
  # End value of lower half
  end_lower = df.shape[0]
  
  # Create a list of upper_half index
  upper_half = [*range(start_upper, end_upper, 1)]
  
  # Create a list of lower_half index
  lower_half = [*range(start_lower, end_lower, 1)]
  
  # Increment the value of lower half by 1
  lower_half = [x.__add__(1) for x in lower_half]
  
  # Combine the two lists
  index_ = upper_half + lower_half
  
  # Update the index of the dataframe
  df.index = index_
  
  # Insert a row at the end
  df.loc[row_number] = row_value
  
  # Sort the index labels
  df = df.sort_index()
  
  # return the dataframe
  return df

#--------------------------------------------------------------------------------
# Trata Vínculos
#--------------------------------------------------------------------------------

def get_vinculos_remuneracoes(page, client):
  global indexes_rel, indexes_rem

  # Filtrando Relações Previdenciárias e removendo colunas
  rel_prev = [line for line in page if line[0].isnumeric()]

  # Pegando colunas Padrões das Relações
  colunas = page[get_colum_index(page, "Seq.")]
  colunas = remove_blank_spaces(colunas)
  # colunas = colunas[:len(colunas) - 12] + "&Indicadores&Remuneracoes"
  colunas = colunas[:len(colunas) - 12] + "&Indicadores"  # Versao normalizada
  indexes_rel = colunas.split("&")

  # Pegando colunas Padrões das Remunerações
  colunas = page[get_colum_index(page, "Competência")]
  colunas = remove_blank_spaces(colunas)
  indexes_rem = colunas.split("&")

  ult_rel: int = 0
  seq: int
  rem_final: list = []
  df_relacao1: list = []
  df_remuneracao: list = []

  # Percorrendo lista de relações
  for n in range(len(rel_prev)):
      clean_up_data_normalized(client, rel_prev[n], df_relacao1, df_remuneracao)

  # Alterando as opções do print do Dataframe
  pd.set_option("display.max_rows", None, "display.max_columns", None)

  # Ajustando vínculos
  if len(df_relacao1) > 0:
    df_relacao1 = pd.concat(df_relacao1, ignore_index=True)
    df_relacao1.drop_duplicates()

    #Helaine e Júlia - definindo o CPF como index de df_relacao
    df_relacao1.insert(0, 'CPF', client['CPF'])
    df_relacao1["CPF"] = df_relacao1['CPF'].ffill()
    df_relacao1=df_relacao1.set_index('CPF')

    # Ajustando remunerações
    df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)
    df_remuneracao = df_remuneracao.set_index('Seq.')

  return df_relacao1, df_remuneracao


def completa_espaços(len_titulo, info):
  if len_titulo > 0 and info != None and not isinstance(info, float):

    qtde_espacos = abs(len_titulo - len(info))
    
    if qtde_espacos > 0:
      for i in range(qtde_espacos-1):
        info = info + ' '
    return info
  return ''

#-------------------------------------------------------------------------------
# Rotina main
#-------------------------------------------------------------------------------

def main():
  global df_remuneracao, seq
  page = doc.get_page(1).lines
  df_relacao = pd.DataFrame()
  df_remuneracao = pd.DataFrame()
  client = get_client(page)

  for pag in range(doc.no_of_pages):
    page2 = doc.get_page(pag+1).lines
    # print(page2, len(page2))

    if len(page2) > 20:
        # Extraindo vínculos e remunerações da página
        df_relacao_td, df_remuneracao_td = get_vinculos_remuneracoes(page2, client)

        # Adicionando vínculos aos demais no DataFrame 
        df_relacao = df_relacao.append(df_relacao_td)

        # Adicionando remunerações as demais no DataFrame
        df_remuneracao = df_remuneracao.append(df_remuneracao_td)

  print("\nRESUMO DO BENEFICIÁRIO")
  print("\nNome do Beneficiário: ", client.get("Nome").values[0])
  print("NIT: ", client.get("NIT").values[0])
  print("CPF: ", client.get("CPF").values[0])
  print("Nome da mãe: ", client.get("Nome da mãe").values[0])
  print("Data de nascimento: ", client.get("Data de nascimento").values[0])
#  print("Sexo: ", client.get("Sexo").values[0])
  print("\n\n")
  print("RESUMO DOS VÍNCULOS")
  
  print("\nCÓDIGO DA EMPRESA", end='\t')
  print("DATA DE INÍCIO NA EMPRESA", end='\t')
  print("DATA FINAL NA EMPRESA", end='\t')
  print("NOME DA EMPRESA", end='\t\t\t\t\t')
  print("TIPO DE VÍNCULO COM A EMPRESA", end='\t')
  print("DATA DA ÚLTIMA REMUNERAÇÃO")

  cod_empresa = ''
  dt_inicio = ''
  dt_fim = ''
  nome_empresa = ''
  tipo_vinculo = ''
  dt_ult_remuneracao = ''
  totalContrib = 0

  umMes = relativedelta(months=+1)


  for i in range(len(df_relacao)):
    tipo_vinculo = df_relacao.get('Tipo Filiado no Vínculo').values[i]

    if not isinstance(tipo_vinculo, float) and tipo_vinculo != None and not 'Empregador' in tipo_vinculo and 'Empregado' in tipo_vinculo:
      cod_empresa = df_relacao.get("Código Emp.").values[i]
      cod_empresa = completa_espaços(25, cod_empresa)
      print(cod_empresa, end='')
      dt_inicio = df_relacao.get("Data Início").values[i]
      dt_inicio = completa_espaços(33, dt_inicio)
      print(dt_inicio, end='')
      dt_fim = df_relacao.get("Data Fim").values[i]
      if(dt_fim != None):
        dt_fim = completa_espaços(25, dt_fim)
        print(dt_fim, end='')
      else:
        print('\t\t\t', end='')
      nome_empresa = df_relacao.get('Origem do Vínculo').values[i]
      nome_empresa = completa_espaços(51, nome_empresa)
      print(nome_empresa, end='')
      tipo_vinculo = completa_espaços(35, tipo_vinculo)
      print(tipo_vinculo, end='')
      dt_ult_remuneracao = df_relacao.get('Últ. Remun.').values[i]
      dt_ult_remuneracao = completa_espaços(34, dt_ult_remuneracao)
      print(dt_ult_remuneracao)
  
  print("\n\nRESUMO DAS CONTRIBUIÇÕES POR EMPRESA")
    
  for i in range(len(df_relacao)):
    tipo_vinculo = df_relacao.get('Tipo Filiado no Vínculo').values[i]
    print('TIPO_VINCULO', tipo_vinculo)
    
    if not isinstance(tipo_vinculo, float) and tipo_vinculo != None and not 'Empregador' in tipo_vinculo and 'Empregado' in tipo_vinculo:

      print('____________________________________________________________________________________________')
      
      nome_empresa = df_relacao.get('Origem do Vínculo').values[i]
      nome_empresa = completa_espaços(47, nome_empresa)
      print("Empresa: ", nome_empresa)
      
      print('Seq.: ', df_relacao.get('Seq.').values[i])
      try:
        df_contr_empresa = df_remuneracao.loc[df_relacao.get('Seq.').values[i]]
        #print('loc_rem', df_contr_empresa)
      except:
        df_contr_empresa = pd.DataFrame()

  #    print(df_relacao.get('Seq.').values[i])
      if len(df_contr_empresa) > 0 and not isinstance(df_contr_empresa.get('Competência'), str):
        print ('len(df_contr_empresa)', df_contr_empresa.shape[0])
        competencia_anterior = datetime.strptime(df_contr_empresa.get('Competência').values[0].strip(), '%m/%Y').date()
        print("\nDATA DA CONTRIBUIÇÃO", end='\t')
        print("VALOR DA CONTRIBUIÇÃO")
        totalContrib += len(df_contr_empresa)

        for j in range(len(df_contr_empresa)):
          competencia_atual = datetime.strptime(df_contr_empresa.get('Competência').values[j].strip(), '%m/%Y').date()
          diferenca_meses = relativedelta(competencia_atual, competencia_anterior)
          diferenca_meses = diferenca_meses.months + (diferenca_meses.years * 12)

          nova_competencia = competencia_anterior              

          for k in range(diferenca_meses-1):
            if (diferenca_meses-1) > 1:
              nova_competencia = nova_competencia + umMes
              df_contr_empresa = df_contr_empresa.append({'Competência': datetime.strftime(nova_competencia, '%m/%Y'), 'Remuneração':'0,00'}, ignore_index = True)
              #actual_index = df_contr_empresa.index.get_loc(datetime.strftime(competencia_atual, '%m/%Y'))
              #actual_index = int(df_contr_empresa.index.get_loc(df_contr_empresa.get('Seq.').values[j]))
              #actual_index = actual_index + 1
              #df_contr_empresa = insert_row(actual_index, df_contr_empresa, {'Competência': datetime.strftime(nova_competencia, '%m/%Y'), 'Remuneração':'0,00'})
              print(df_contr_empresa.get('Competência').values[j+k], end='\t\t\t')
              print(df_contr_empresa.get('Remuneração').values[j+k])

          
          print(competencia_atual.strftime('%m/%Y'), end='\t\t\t')
          print(df_contr_empresa.get('Remuneração').values[j])
          competencia_anterior = competencia_atual
      elif isinstance(df_contr_empresa.get('Competência'), str):
        print("\nDATA DA CONTRIBUIÇÃO", end='\t')
        print("VALOR DA CONTRIBUIÇÃO")
        print(df_contr_empresa.get('Competência'), end='\t\t\t')
        print(df_contr_empresa.get('Remuneração'))
    
    
      print("\nQUANTIDADE DE CONTRIBUIÇÕES PELA EMPRESA: ", len(df_contr_empresa))
      if len(df_contr_empresa) > 0:
        if not isinstance(df_contr_empresa.get('Competência'), str):
          print('PERÍODO DAS CONTRIBUIÇÕES: ', df_contr_empresa.get('Competência').values[0], 'a', df_contr_empresa.get('Competência').values[len(df_contr_empresa)-1])
        else:
          print('PERÍODO DAS CONTRIBUIÇÕES: ', df_contr_empresa.get('Competência'))

  print("\nQUANTIDADE TOTAL DE CONTRIBUIÇÕES: ", totalContrib)
  
  client.to_excel('beneficiario.xlsx',index=None)
  df_relacao.to_excel('vinculos.xlsx',index=None)
  df_remuneracao.to_excel('remuneracoes.xlsx',index=None)

main()

#-------------------------------------------------------------------------------
# Export dos dados
#-------------------------------------------------------------------------------

print('\n')
print(Fore.BLUE + '... Exportando os dados.' + fn)
print('\n')
print(Style.RESET_ALL)

for f in ['/content/beneficiario.xlsx','/content/vinculos.xlsx','/content/remuneracoes.xlsx']:
  df = pd.read_excel(f)
  display(df.head())
  print('\n\n')
  files.download(f)

#-------------------------------------------------------------------------------
# Cálculo Final
#-------------------------------------------------------------------------------

competencia_atual = datetime.strptime('07/1990', '%m/%Y').date()

competencia_anterior = datetime.strptime('03/1990', '%m/%Y').date()

diferenca_meses = relativedelta(competencia_atual, competencia_anterior)
diferenca_meses = diferenca_meses.months + (diferenca_meses.years * 12)

nova_competencia = competencia_anterior

for i in range(diferenca_meses-1):
  umMes = relativedelta(months=+1)
  nova_competencia = nova_competencia + umMes
  print(datetime.strftime(nova_competencia, '%m/%Y'))

print('DIFERENÇA', diferenca_meses)


